<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/CV19Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ str(device))

In [0]:
from google.colab import drive
import os.path
from os import path

drive.mount('/content/drive')
PATH = '/content/drive/My Drive/covid'
MODEL_DIR =  PATH +'/models'
INPUT_DIR = PATH + '/input'
VALIDATION_DATA_DIR = '/input/validation'
TRAINING_DATA_DIR = '/input/training'

In [0]:
from torch_geometric.data import Data, DataLoader, InMemoryDataset

class CovidDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(CovidDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
      return ['']#left this to be empty to avoid accidential processing

    @property
    def processed_file_names(self):
      print(self.root)
      return ['processed.dt']

    def download(self):
      pass
    
    def process(self):
      pass

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size)

        self.layers = dict()

        
        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size)
        
    def forward(self, input, hidden):
      output = self.fc1(input)

      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.layers['fc_'+str(i+2)](output)
        k +=2

      output = self.fc2(output)

      return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





Attention Decoder
^^^^^^^^^^^^^^^^^

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [0]:
MAX_LENGTH = 20
class AttnDecoderRNN(nn.Module):
  def __init__(self, input_size, parameter_sizes,repeats ,output_size, dropout_p=0.1, max_length=MAX_LENGTH):
    super(AttnDecoderRNN, self).__init__()
    
    self.input_size = input_size
    self.repeater_input_size = parameter_sizes[0]
    self.hidden_size = parameter_sizes[1]
    self.repeats = repeats
    self.output_size = output_size
    self.dropout_p = dropout_p
    self.max_length = max_length

    self.fc1 = nn.Linear(self.input_size, self.hidden_size)
    self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)

    self.layers = dict()
    k = 0
    for i in range(repeats):
      i = i+k
      self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
      self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
      self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
      k+=2

    self.module_list = nn.ModuleDict(self.layers)

    self.out =  nn.Linear(self.repeater_input_size, output_size)
    #nn.Linear(self.hidden_size, self.output_size)

  def forward(self, input,  hidden, encoder_outputs):
    output = self.fc1(input)
    embedded = self.dropout(output)
    
    hidden[0] = hidden[0].squeeze()
    attn_weights = F.softmax(
        self.attn(torch.cat((embedded, hidden[0]), 1)), dim=1) #embedded[0]
    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                              encoder_outputs.unsqueeze(0))
    
    output = torch.cat((embedded, attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)
    hidden[0] = hidden[0].unsqueeze(0)

    k = 0
    for i in range(self.repeats):
      i = i+k
      output = self.layers['fc_'+str(i)](output)
      output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
      output = self.layers['fc_'+str(i+2)](output)
      k +=2

    output = self.out(output[0])
    output = F.log_softmax(output, dim=1)

    return output, hidden, attn_weights

  def initHidden(self, batch_size):
    return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [0]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
  

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'encoder repeats' : model_parameters_dict[i][0]['repeats'],
                        'decoder repeats' : model_parameters_dict[i][1]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

In [0]:
#Generate the list of repeat count parameters list
def generate_repeat_counts(alphas , training_dataset_length, enc_repeated_nn_parameters = 132096, dec_repeated_nn_parameters = 132096):
  repeats = []
  for alpha in alphas:
    preferred_parameters  = (1/alpha)* (training_dataset_length / (feature_len+yhat_size))
    for i in range(1,100):
      dec_repeats = i
      enc_repeats = int((preferred_parameters - 
                  (preferred_parameters%enc_repeated_nn_parameters) -
                    (dec_repeated_nn_parameters*dec_repeats)
                  )/ enc_repeated_nn_parameters)
      model_id = 0
      for j in range(1, enc_repeats+1):
        repeats.append((i, j))
  return repeats

In [0]:
def generate_model_parameters(repeat_counts, feature_len, output_len):
  model_dict = dict()
  model_id = 0
  for dec_repeat, enc_repeat in repeat_counts:
    model_dict[model_id] = [{
         'input_size': feature_len,
         'parameter_sizes': [128, 128] ,
         'repeats' : enc_repeat ,
         'output_size' : 128
        },
        {
            'input_size' : 1, 
            'parameter_sizes' : [128, 128],
            'repeats' : dec_repeat ,
            'output_size': output_len,
            'dropout_p' : 0.1, 
        }]
    model_id+=1
  return model_dict

#generate_model_parameters(generate_repeat_counts())

In [0]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,batch_size, max_length=MAX_LENGTH):
  input_tensor = input_tensor.type(torch.FloatTensor)
  target_tensor= target_tensor.type(torch.FloatTensor)

  encoder_hidden = []
  for i in range(encoder.repeats):
    encoder_hidden.append(encoder.initHidden(batch_size))

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  
  input_length = input_tensor.size(1)
  target_length = target_tensor.size(1)

  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

  loss = 0
  
  for ei in range(input_sequence_len):
    input_tensor_seq = input_tensor.view(input_sequence_len,batch_size, feature_len)[ei]
    input_tensor_seq = input_tensor_seq.view(1, batch_size, feature_len).to(device)

    encoder_output, encoder_hidden = encoder(
        input_tensor_seq, encoder_hidden)
    encoder_outputs[ei] = encoder_output[0, 0]

  decoder_input = torch.ones((batch_size,1), device=device).type(torch.FloatTensor)

  decoder_hidden = encoder_hidden  
  for i in range(decoder.repeats):
    decoder_hidden.append(decoder.initHidden(batch_size))

  for di in range(target_length):
    target_tensor_seq = target_tensor.view(output_sequence_len, batch_size, yhat_size)
    decoder_input = decoder_input.type(torch.FloatTensor).to(device)
    decoder_output, decoder_hidden, decoder_attention = decoder(
        decoder_input, decoder_hidden, encoder_outputs)
    topv, topi = decoder_output.topk(1)
    decoder_input = topi.detach()  # detach from history as input
    loss += criterion(decoder_output, target_tensor_seq[di].to(device))

  loss.backward()
  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item() / target_length

In [0]:
def evaluate(input_tensor, target_tensor, encoder, decoder, criterion,batch_size, max_length=MAX_LENGTH):
  with torch.no_grad():
    input_tensor = input_tensor.type(torch.FloatTensor)
    target_tensor= target_tensor.type(torch.FloatTensor)

    encoder_hidden = []
    for i in range(encoder.repeats):
      encoder_hidden.append(encoder.initHidden(batch_size))

    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    validation_loss = 0
    
    for ei in range(input_sequence_len):
      input_tensor_seq = input_tensor.view(input_sequence_len,batch_size, feature_len)[ei]
      input_tensor_seq = input_tensor_seq.view(1, batch_size, feature_len).to(device)

      encoder_output, encoder_hidden = encoder(
          input_tensor_seq, encoder_hidden)
      encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.ones((batch_size,1), device=device).type(torch.FloatTensor)

    decoder_hidden = encoder_hidden  
    for i in range(decoder.repeats):
      decoder_hidden.append(decoder.initHidden(batch_size))


    for di in range(target_length):
      target_tensor_seq = target_tensor.view(output_sequence_len, batch_size, yhat_size)
      decoder_input = decoder_input.type(torch.FloatTensor).to(device)
      decoder_output, decoder_hidden, decoder_attention = decoder(
          decoder_input, decoder_hidden, encoder_outputs)
      topv, topi = decoder_output.topk(1)
      decoder_input = topi.detach()  # detach from history as input
      validation_loss += criterion(decoder_output, target_tensor_seq[di].to(device))

  return validation_loss.item() / target_length

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [0]:
def train_batch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/input_sequence_len)
    loss += (train(batch.x.view(batch_size, input_sequence_len , feature_len), batch.y, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def batch_evaluate(dataloader, encoder, decoder, criterion, output_sequence_len):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/input_sequence_len)
    loss += (evaluate(batch.x.view(batch_size, input_sequence_len , feature_len), batch.y, encoder,
                     decoder, criterion, batch_size))/ batch_size
  return loss / len(dataloader)


This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [0]:
def trainValidationIters(train_dataloader,validation_dataloader, encoder, decoder,  encoder_optimizer, 
                         decoder_optimizer, criterion ,n_iters, output_sequence_len,input_sequence_len,
                         current_model_dir, model_id,session_id,print_every=1000, plot_every=100,save_every = 100):
  #yhat_size = 2
  #input_sequence_len = 10

  start = time.time()
  plot_losses = []
  
  # Reset every print_every # Reset every plot_every
  print_training_loss_total = 0
  plot_training_loss_total = 0
  print_validation_loss_total = 0
  plot_validation_loss_total = 0
  save_validation_loss_total = 0
  save_training_loss_total = 0
      
  losses = []

  for iter in range(1, n_iters + 1):
      training_loss = train_batch(train_dataloader, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion )

      validation_loss = batch_evaluate(validation_dataloader, encoder,
                    decoder, criterion,output_sequence_len )
      
      print_training_loss_total += training_loss
      plot_training_loss_total += training_loss
      print_validation_loss_total += validation_loss
      plot_validation_loss_total += validation_loss
      
      save_training_loss_total += training_loss
      save_validation_loss_total += validation_loss
      
      if iter % print_every == 0:
          print_training_loss_avg = print_training_loss_total / print_every
          print_training_loss_total = 0
          print_validation_loss_avg = print_validation_loss_total / print_every
          print_validation_loss_total = 0
          print('%s (%d %d%%) training:%.4f validation:%.4f' % (timeSince(start, iter / n_iters),
                                        iter, iter / n_iters * 100, print_training_loss_avg,print_validation_loss_avg, ))
          print('train/validation difference : ',print_validation_loss_avg-print_training_loss_avg)

      #if iter % plot_every == 0:
      #    plot_training_loss_avg = plot_training_loss_total / plot_every
      #    plot_validation_loss_avg = plot_validation_loss_total / plot_every
      #    plot_training_loss.append(plot_training_loss_avg)
      #    plot_validation_loss.append(plot_validation_loss_avg)
      #    showPlot(plot_training_losses,plot_validation_loss)
      #    plot_training_loss_total = 0
      #    plot_validation_loss_total
        
      if iter % save_every == 0:
        save_training_loss_avg = save_training_loss_total / save_every
        save_validation_loss_avg = plot_validation_loss_total / save_every
        
        #'arch': args.arch,
        save_checkpoint({
            'epoch': iter + 1,
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'training_loss': save_training_loss_avg,
            'validation_loss': save_validation_loss_avg,
            'encoder_optimizer' : encoder_optimizer.state_dict(),
            'decoder_optimizer' : decoder_optimizer.state_dict(),
        }, False, filename=current_model_dir+'/'+str(iter)+'_'+str(session_id)+'.pt')
      losses.append([training_loss, validation_loss])

  save_dict({
      'losses' : losses
  }, filename=current_model_dir+'/losses_'+session_id)

  return losses

In [0]:
input_sequence_len = 10
output_sequence_len = 20
training_batch_size = 8192 # for production 131072
validation_batch_size = 2048
yhat_size = 2
feature_len = 8

training_dataset = CovidDataSet(PATH+TRAINING_DATA_DIR, input_sequence_len, output_sequence_len)
training_dataset = training_dataset.shuffle()
training_dataset = training_dataset[:131072]
training_dataloader = DataLoader(training_dataset,batch_size=training_batch_size)

validation_dataset = CovidDataSet(PATH+VALIDATION_DATA_DIR, input_sequence_len, output_sequence_len)
validation_dataset = validation_dataset.shuffle()
validation_dataloader = DataLoader(validation_dataset,batch_size=validation_batch_size)

print('validation :', len(validation_dataloader), 'training : ', len(training_dataloader))

In [0]:
session_id =str(1001)+ '_'+str(datetime.now(tz=None).microsecond)
repeat_counts = generate_repeat_counts([1,2,3], 3*len(training_dataset))

model_configurations = generate_model_parameters(repeat_counts, 8, 2)
save_model_parameters(model_configurations, fileName= PATH + '/model_configurations_'+session_id+'.csv')
models = dict()
len(model_configurations)
len(repeat_counts)

In [0]:
#Run all the candidate models
learning_rate = 0.1
for i in model_configurations:
  if i < 10:
    continue
  print('training :',i)
  #instantiate the models
  current_model_dir =  MODEL_DIR +'/'+str(i)
  if os.path.exists(current_model_dir) == False:
    os.mkdir(current_model_dir)
  
  encoder = EncoderRNN(**model_configurations[i][0]).to(device)
  decoder = AttnDecoderRNN(**model_configurations[i][1]).to(device)
  encoder_optimizer= optim.Adam(encoder.parameters(), lr=learning_rate)
  decoder_optimizer= optim.Adam(decoder.parameters(), lr=learning_rate)
  criterion = nn.MSELoss()

  losses = trainValidationIters(training_dataloader,validation_dataloader,
                    encoder, decoder,
                    encoder_optimizer, decoder_optimizer,
                    criterion, n_iters = 100,output_sequence_len=output_sequence_len,
                    input_sequence_len=input_sequence_len,
                    current_model_dir=current_model_dir,  model_id = i,session_id = session_id,
                     print_every=10, save_every = 10)
  models[i] = {
      'encoder' : encoder.state_dict(),
      'decoder' : decoder.state_dict(),
      'encoder_optimizer' : encoder_optimizer.state_dict(),
      'decoder_optimizer' : decoder_optimizer.state_dict(),
      'losses' : losses
  }
  print('Model ',str(i), ': trained and evaluated')

save_dict(models, MODEL_DIR+'/models_'+session_id+'.pt')

training : 3
3m 33s (- 32m 1s) (10 10%) training:173.4740 validation:2190.2324
train/validation difference :  2016.7583596612699
7m 6s (- 28m 26s) (20 20%) training:170.9638 validation:2177.7728
train/validation difference :  2006.8089751082975
10m 41s (- 24m 57s) (30 30%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090948874492
14m 15s (- 21m 23s) (40 40%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091080481547
17m 49s (- 17m 49s) (50 50%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090967471142
21m 23s (- 14m 15s) (60 60%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091005141275
24m 57s (- 10m 41s) (70 70%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090756232277
28m 32s (- 7m 8s) (80 80%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090975577372
32m 6s (- 3m 34s) (90 90%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090999896065
35m 40s (- 0m 0s) (100 100%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091089064615
Model  3 : trained and evaluated
training : 4
3m 45s (- 33m 48s) (10 10%) training:173.5196 validation:2202.3471
train/validation difference :  2028.8275353356416
7m 30s (- 30m 2s) (20 20%) training:170.9638 validation:2177.7730
train/validation difference :  2006.809155901524
11m 14s (- 26m 14s) (30 30%) training:170.9638 validation:2177.7729
train/validation difference :  2006.809106283857
14m 59s (- 22m 29s) (40 40%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091004187602
18m 45s (- 18m 45s) (50 50%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091198737163
22m 30s (- 15m 0s) (60 60%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091135317818
26m 15s (- 11m 15s) (70 70%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090731507014
30m 1s (- 7m 30s) (80 80%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091096240596
33m 46s (- 3m 45s) (90 90%) training:170.9638 validation:2177.7729
train/validation difference :  2006.809140234663
37m 31s (- 0m 0s) (100 100%) training:170.9638 validation:2177.7730
train/validation difference :  2006.8091474825876
Model  4 : trained and evaluated
training : 5
3m 56s (- 35m 29s) (10 10%) training:172.8378 validation:2179.0659
train/validation difference :  2006.228061978189
7m 52s (- 31m 31s) (20 20%) training:170.9638 validation:2177.7729
train/validation difference :  2006.809092407896
11m 49s (- 27m 35s) (30 30%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090995127695
15m 47s (- 23m 41s) (40 40%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8090824466838
19m 45s (- 19m 45s) (50 50%) training:170.9638 validation:2177.7729
train/validation difference :  2006.8091159159676

In [0]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def save_dict(dictionary, filename):
  np.save(filename, dictionary) 

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points, points2):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.plot(points)
    plt.plot(points2)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [0]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [0]:
evaluateRandomly(encoder1, attn_decoder1)

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [0]:
""""output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())
"""

For a better viewing experience we will do the extra work of adding axes
and labels:




In [0]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

Exercises
=========

-  Try with a different dataset

   -  Another language pair
   -  Human → Machine (e.g. IOT commands)
   -  Chat → Response
   -  Question → Answer

-  Replace the embeddings with pre-trained word embeddings such as word2vec or
   GloVe
-  Try with more layers, more hidden units, and more sentences. Compare
   the training time and results.
-  If you use a translation file where pairs have two of the same phrase
   (``I am test \t I am test``), you can use this as an autoencoder. Try
   this:

   -  Train as an autoencoder
   -  Save only the Encoder network
   -  Train a new Decoder for translation from there


